In [231]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
sns.set()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [235]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df_with_rolling_features.pkl')

In [240]:
df = df[df['total_points'] >= 0] #possible, but rare, to get negative points. Will remove negative points for ease of use
df['past_three_avg_mins'] = (df['minutes_prev'] + df['minutes_prev_2'] + df['minutes_prev_3'])/3
df = df[df['past_three_avg_mins'] > 60]
df = df.drop(columns = 'past_three_avg_mins')

In [241]:
def prep_final_features(df):
    df['was_home'] = df['was_home'].astype('int') #convert bool to numeric
    positions_df = pd.get_dummies(df['position'], prefix='position_')
    df = df.join(positions_df)
    return df

In [242]:
df = prep_final_features(df)

In [243]:
metadata_cols = ['player', 'team', 'position','opponent_team', 'gw', 'kickoff_time', 'season', 'team_opponent','total_points_opponent']

In [244]:
metadata = df[metadata_cols]

In [245]:
df = df.dropna()

In [246]:
X = df.drop(columns = metadata_cols)
X = X.drop(columns = 'total_points')

In [247]:
y = df['total_points']

In [248]:
#save the final features names -- I'll use this when predicting future data to make sure I have the correct features
with open('/Users/andrewpeters/GitHub/fpl/data/interim/x_features.pkl', 'wb') as f:
    pickle.dump(list(X), f)

In [257]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [259]:
with open('/Users/andrewpeters/GitHub/fpl/data/processed/X.npy', 'wb') as file:
    np.save(file, X)

In [260]:
with open('/Users/andrewpeters/GitHub/fpl/data/processed/y.npy', 'wb') as file:
    np.save(file, y)

In [261]:
X.shape

(32904, 320)